# Dados do Setor de Saúde

Este notebook tem como objetivo analisar os indicadores mais importantes ou principais sobre o setor de saúde suplementar, sendo eles principalmente: sinistralidade, taxa de cobertura, número de operadores e VCHM.

In [7]:
import pandas as pd
import numpy as np

import src.demonstracoes_contabeis as dc

# options
pd.options.display.float_format = '{:.2f}'.format

## Sinistralidade

Segundo informações divulgada pela ANS, o cálculo da SINISTRALIDADE deve seguir a seguinte fórmula:

<br>

$$
Sinistralidade = \frac{\text{Eventos Indenizáveis Líquidos} + |CCT|}{\text{Receitas com Operações de Assistência a Saúde} + |CCT|}
$$

<br>

Essas informações nos demonstrativos contábeis correspondem aos códigos:

- Eventos Indenizáveis Líquidos - 41
- Receitas com Operações de Assistência a Saúde - 31
- Contraprestações de Corresponsabilidade Cedida (CCT) - 3117, 31171

In [2]:
data = dc.download_data(start_year=2020, end_year=2023)
data = data.drop(['DESCRICAO', 'VL_SALDO_INICIAL'], axis=1)

data.info()

array(['2021-10-01', '2023-01-01', '01/01/2021', '01/04/2022',
       '2023-04-01', '01/07/2021', '01/07/2020', '01/04/2020',
       '01/10/2020', '2022/01/01', '2022-07-01', '01/04/2021',
       '01/01/2020', '2022-10-01'], dtype=object)

In [11]:
# seleciona somente as linhas relevantes do demonstrativo contabil e aplica módulo
ans_contabil = data.loc[data['CD_CONTA_CONTABIL'].isin([3117, 31171, 41, 31])].copy()

ans_contabil['VL_SALDO_FINAL'] = ans_contabil['VL_SALDO_FINAL'].apply(lambda x: float(x.replace(',', '.')))
ans_contabil['VL_SALDO_FINAL'] = np.where(ans_contabil['CD_CONTA_CONTABIL'].isin([3117, 31171]), ans_contabil['VL_SALDO_FINAL'].abs(), ans_contabil['VL_SALDO_FINAL'])

ans_contabil.sample(10).reset_index(drop=True)

,DATA,REG_ANS,CD_CONTA_CONTABIL,VL_SALDO_FINAL
0,01/04/2020,418129,31,2359126.67
1,01/07/2021,422606,31171,0.00
2,01/07/2020,410624,31,8502091.16
3,2021-10-01,407097,41,4807929.30
4,01/01/2020,346659,41,1190532632.73
5,2022-07-01,303267,31,282008124.86
6,2022-10-01,349712,31171,1008890.97
7,01/04/2020,418277,31,833637.21
8,01/10/2020,312282,31,35892384.20
9,2022/01/01,403911,31,287787831.11


In [23]:
# limpeza das datas no banco de dados
def parse_dates(str_date: str):
    for format in ('%Y-%m-%d', '%Y/%m/%d', '%d/%m/%Y'):
        try:
            return pd.to_datetime(str_date, format=format)
        except ValueError:
            continue
    return pd.NaT

ans_contabil['DATA'] = ans_contabil['DATA'].apply(parse_dates)

months = ans_contabil['DATA'].dt.month
years = ans_contabil['DATA'].dt.year

cond = [months == 1, months == 4, months == 7, months == 10]
labels = ['1T' + years.astype(str), '2T' + years.astype(str), '3T' + years.astype(str), '4T' + years.astype(str)]

ans_contabil['TRIMESTRE'] = np.select(cond, labels, default=None)

ans_contabil['TRIMESTRE'].unique()

array(['4T2021', '1T2023', '1T2021', '2T2022', '2T2023', '3T2021',
       '3T2020', '2T2020', '4T2020', '1T2022', '3T2022', '2T2021',
       '1T2020', '4T2022'], dtype=object)

In [107]:
ans_contabil.reset_index(drop=True)

,DATA,REG_ANS,CD_CONTA_CONTABIL,VL_SALDO_FINAL,TRIMESTRE
0,2021-10-01,422975,41,0.00,4T2021
1,2021-10-01,422991,31,0.00,4T2021
2,2021-10-01,422991,3117,0.00,4T2021
3,2021-10-01,422991,31171,0.00,4T2021
4,2021-10-01,422614,31,567286.84,4T2021
...,...,...,...,...,...
34834,2022-10-01,338346,41,18517185.54,4T2022
34835,2022-10-01,340162,41,4080982.56,4T2022
34836,2022-10-01,342131,31,39985307.37,4T2022
34837,2022-10-01,342386,41,26590828.14,4T2022


In [105]:
# pivot

ans_pivot = ans_contabil.pivot_table(index='TRIMESTRE', columns='CD_CONTA_CONTABIL', values='VL_SALDO_FINAL', aggfunc='sum')

ans_pivot

CD_CONTA_CONTABIL,31,41,3117,31171
TRIMESTRE,,,,
1T2020,56500059157.66,43357020234.91,3938081333.67,3881691726.41
1T2021,60287465821.36,47115660112.64,3567627838.72,3511469091.39
1T2022,60266234667.89,49924983149.61,785399355.24,737828831.15
1T2023,67766709958.01,57067593162.44,533814326.29,499923928.90
2T2020,112165488755.88,78167388881.27,7170344430.86,7065719293.80
2T2021,120006452911.72,97597769830.96,7451296782.34,7338488109.23
2T2022,121161630424.84,103844054330.65,1584007090.42,1495306592.55
2T2023,136861410762.26,116413163841.56,1118867793.62,1049213521.56
3T2020,169036747048.80,120174456304.81,10545111579.19,10383258092.49
